## Where is January 2025 data?

In [1]:
import _section1_utils
import _section2_utils
import geopandas as gpd
import merge_data
import numpy as np
import pandas as pd
from segment_speed_utils import gtfs_schedule_wrangling
from shared_utils import rt_dates
from update_vars import GTFS_DATA_DICT, RT_SCHED_GCS, SCHED_GCS, SEGMENT_GCS

In [2]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [3]:
analysis_date_list = rt_dates.y2025_dates

In [4]:
analysis_date_list_all = (
    rt_dates.y2024_dates + rt_dates.y2023_dates + rt_dates.y2025_dates
)

In [17]:
schd_vp_url = f"{GTFS_DATA_DICT.digest_tables.dir}{GTFS_DATA_DICT.digest_tables.route_schedule_vp}.parquet"

In [18]:
og_df = pd.read_parquet(schd_vp_url)

In [19]:
og_df.columns

Index(['schedule_gtfs_dataset_key', 'direction_id', 'time_period',
       'avg_scheduled_service_minutes', 'avg_stop_miles', 'n_scheduled_trips',
       'frequency', 'is_express', 'is_rapid', 'is_rail', 'is_coverage',
       'is_downtown_local', 'is_local', 'service_date', 'typology',
       'minutes_atleast1_vp', 'minutes_atleast2_vp',
       'total_rt_service_minutes', 'total_scheduled_service_minutes',
       'total_vp', 'vp_in_shape', 'is_early', 'is_ontime', 'is_late',
       'n_vp_trips', 'vp_per_minute', 'pct_in_shape',
       'pct_rt_journey_atleast1_vp', 'pct_rt_journey_atleast2_vp',
       'pct_sched_journey_atleast1_vp', 'pct_sched_journey_atleast2_vp',
       'rt_sched_journey_ratio', 'avg_rt_service_minutes', 'sched_rt_category',
       'speed_mph', 'route_long_name', 'route_short_name',
       'route_combined_name', 'route_id', 'base64_url',
       'organization_source_record_id', 'organization_name',
       'caltrans_district', 'route_primary_direction',
       'schedule

In [23]:
all_ops_jan_only = og_df.loc[og_df.service_date == "2025-01-15T00:00:00.000000000"]

In [38]:
all_ops_jan_only.service_date.unique()

array(['2025-01-15T00:00:00.000000000'], dtype='datetime64[ns]')

In [24]:
all_ops_jan_only.time_period.value_counts()

all_day    5280
peak       4006
offpeak    3749
Name: time_period, dtype: int64

In [42]:
all_ops_jan_only[
    [
        "organization_name",
        "route_id",
        "time_period",
        "is_early",
        "is_ontime",
        "is_late",
    ]
].sample(10)

,organization_name,route_id,time_period,is_early,is_ontime,is_late
133066,Sacramento Regional Transit District,102,None,0,0,0
142540,City and County of San Francisco,14,all_day,5,20,122
328353,Alameda-Contra Costa Transit District,200,offpeak,0,1,23
179054,Peninsula Corridor Joint Powers Board,Local Weekday,None,0,0,0
70440,Los Angeles County Metropolitan Transportation Authority,802,all_day,0,4,85
152167,San Joaquin Regional Transit District,510,all_day,0,8,14
43118,City of Ojai,4251,all_day,0,0,2
331874,Stanislaus Regional Transit Authority,45,peak,0,0,6
131558,Tahoe Truckee Area Regional Transportation,TLR,None,0,0,0
257478,Livermore-Amador Valley Transit Authority,1,None,0,0,0


### In the original dataframe, not loaded using the function in  `_merge_data.section_2_utils()` there are peak/offpeak values in `time_period`
* There should only be 6 rows for each route (ideally)
* Dir 1: all day, peak, offpeak
* Dir 0: all day, peak, offpeak.

In [48]:
ac_transit_og_one_route = all_ops_jan_only.loc[
    (all_ops_jan_only.organization_name == "Alameda-Contra Costa Transit District")
    & (all_ops_jan_only.route_id == "200")
]

In [54]:
ac_transit_og_one_route.shape

(10, 46)

In [55]:
ac_transit_og_one_route.head(2)

,schedule_gtfs_dataset_key,direction_id,time_period,avg_scheduled_service_minutes,avg_stop_miles,n_scheduled_trips,frequency,is_express,is_rapid,is_rail,is_coverage,is_downtown_local,is_local,service_date,typology,minutes_atleast1_vp,minutes_atleast2_vp,total_rt_service_minutes,total_scheduled_service_minutes,total_vp,vp_in_shape,is_early,is_ontime,is_late,n_vp_trips,vp_per_minute,pct_in_shape,pct_rt_journey_atleast1_vp,pct_rt_journey_atleast2_vp,pct_sched_journey_atleast1_vp,pct_sched_journey_atleast2_vp,rt_sched_journey_ratio,avg_rt_service_minutes,sched_rt_category,speed_mph,route_long_name,route_short_name,route_combined_name,route_id,base64_url,organization_source_record_id,organization_name,caltrans_district,route_primary_direction,schedule_source_record_id,name
224341,c499f905e33929a641f083dad55c521e,0.00,all_day,51.62,9.33,45,1.88,0.00,0.00,0.00,1.00,1.00,0.00,2025-01-15,downtown_local,3122,3080,5886.62,2323.00,8863,8821,0,1,44,45,1.51,0.99,0.53,0.52,1.00,1.00,2.53,130.81,schedule_and_vp,16.91,Decoto - Newark Blvd. - Mowry,200,200 Decoto - Newark Blvd. - Mowry,200,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZWVkcz9vcGVyYXRvcl9pZD1BQw==,recOZgevYf7Jimm9L,Alameda-Contra Costa Transit District,04 - Oakland,Southbound,recJjD8JT53sK302o,Bay Area 511 AC Transit Schedule
224405,c499f905e33929a641f083dad55c521e,0.00,None,51.62,9.33,24,1.00,0.00,0.00,0.00,1.00,1.00,0.00,2025-01-15,downtown_local,0,0,NaN,NaN,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,schedule_only,NaN,Decoto - Newark Blvd. - Mowry,200,200 Decoto - Newark Blvd. - Mowry,200,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZWVkcz9vcGVyYXRvcl9pZD1BQw==,recOZgevYf7Jimm9L,Alameda-Contra Costa Transit District,04 - Oakland,Southbound,recJjD8JT53sK302o,Bay Area 511 AC Transit Schedule


In [49]:
ac_transit_og_one_route.time_period.value_counts()

all_day    2
offpeak    2
peak       2
Name: time_period, dtype: int64

In [52]:
ac_transit_og_one_route.sched_rt_category.unique()

['schedule_and_vp', 'schedule_only', 'vp_only']
Categories (3, object): ['schedule_only', 'vp_only', 'schedule_and_vp']

In [51]:
ac_transit_og_one_route.columns

Index(['schedule_gtfs_dataset_key', 'direction_id', 'time_period',
       'avg_scheduled_service_minutes', 'avg_stop_miles', 'n_scheduled_trips',
       'frequency', 'is_express', 'is_rapid', 'is_rail', 'is_coverage',
       'is_downtown_local', 'is_local', 'service_date', 'typology',
       'minutes_atleast1_vp', 'minutes_atleast2_vp',
       'total_rt_service_minutes', 'total_scheduled_service_minutes',
       'total_vp', 'vp_in_shape', 'is_early', 'is_ontime', 'is_late',
       'n_vp_trips', 'vp_per_minute', 'pct_in_shape',
       'pct_rt_journey_atleast1_vp', 'pct_rt_journey_atleast2_vp',
       'pct_sched_journey_atleast1_vp', 'pct_sched_journey_atleast2_vp',
       'rt_sched_journey_ratio', 'avg_rt_service_minutes', 'sched_rt_category',
       'speed_mph', 'route_long_name', 'route_short_name',
       'route_combined_name', 'route_id', 'base64_url',
       'organization_source_record_id', 'organization_name',
       'caltrans_district', 'route_primary_direction',
       'schedule

In [58]:
all_ops_jan_only.loc[
    (all_ops_jan_only.organization_name == "City and County of San Francisco")].route_id.unique()

array(['1', '12', '14', '14R', '18', '19', '1X', '2', '21', '22', '23',
       '24', '27', '28', '28R', '29', '30', '31', '33', '35', '36', '37',
       '38', '38R', '39', '43', '44', '45', '48', '49', '5', '52', '54',
       '55', '56', '57', '58', '5R', '6', '66', '67', '7', '714', '8',
       '8AX', '8BX', '9', '90', '9R', 'CA', 'F', 'FBUS', 'J', 'K', 'KBUS',
       'KLM', 'L', 'LBUS', 'LOWL', 'M', 'N', 'NBUS', 'NOWL', 'PH', 'PM',
       '15', '25', '91', 'T', 'TBUS'], dtype=object)

In [59]:
sf_og_one_route = all_ops_jan_only.loc[
    (all_ops_jan_only.organization_name == "City and County of San Francisco")
    & (all_ops_jan_only.route_id == "22")
]

In [61]:
sf_og_one_route.head(1)

,schedule_gtfs_dataset_key,direction_id,time_period,avg_scheduled_service_minutes,avg_stop_miles,n_scheduled_trips,frequency,is_express,is_rapid,is_rail,is_coverage,is_downtown_local,is_local,service_date,typology,minutes_atleast1_vp,minutes_atleast2_vp,total_rt_service_minutes,total_scheduled_service_minutes,total_vp,vp_in_shape,is_early,is_ontime,is_late,n_vp_trips,vp_per_minute,pct_in_shape,pct_rt_journey_atleast1_vp,pct_rt_journey_atleast2_vp,pct_sched_journey_atleast1_vp,pct_sched_journey_atleast2_vp,rt_sched_journey_ratio,avg_rt_service_minutes,sched_rt_category,speed_mph,route_long_name,route_short_name,route_combined_name,route_id,base64_url,organization_source_record_id,organization_name,caltrans_district,route_primary_direction,schedule_source_record_id,name
143279,7cc0cb1871dfd558f11a2885c145d144,0.00,all_day,46.87,4.75,186,7.75,0.00,0.00,0.00,0.00,1.00,0.00,2025-01-15,downtown_local,9864,9719,13857.67,8658.00,29100,28584,3,80,99,182,2.10,0.98,0.71,0.70,1.00,1.00,1.60,76.14,schedule_and_vp,5.78,FILLMORE,22,22 FILLMORE,22,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZWVkcz9vcGVyYXRvcl9pZD1TRg==,rechaapWbeffO33OX,City and County of San Francisco,04 - Oakland,Southbound,recHD22phgJs34JHP,Bay Area 511 Muni Schedule


In [60]:
sf_og_one_route[["sched_rt_category","time_period"]]

,sched_rt_category,time_period
143279,schedule_and_vp,all_day
143318,schedule_only,None
143319,schedule_only,None
143339,schedule_and_vp,all_day
143378,schedule_only,None
143379,schedule_only,None
321225,vp_only,offpeak
321226,vp_only,peak
321227,vp_only,offpeak
321228,vp_only,peak


### Something has gone wrong with the merges that I fixed awhile back.

In [53]:
ac_transit_og_one_route[["sched_rt_category", "time_period"]]

,sched_rt_category,time_period
224341,schedule_and_vp,all_day
224405,schedule_only,None
224406,schedule_only,None
224428,schedule_and_vp,all_day
224471,schedule_only,None
224472,schedule_only,None
328353,vp_only,offpeak
328354,vp_only,peak
328355,vp_only,offpeak
328356,vp_only,peak


In [43]:
ac_transitonly = _section2_utils.load_schedule_vp_metrics(
    "Alameda-Contra Costa Transit District"
)

In [44]:
ac_jan_only = ac_transitonly.loc[ac_transitonly.Date == "2025-01-15T00:00:00.000000000"]

In [45]:
ac_jan_only.head(2)

,schedule_gtfs_dataset_key,dir_0_1,Period,Average Scheduled Service (trip minutes),Average Stop Distance (miles),# scheduled trips,Trips per Hour,is_express,is_rapid,is_rail,is_coverage,is_downtown_local,is_local,Date,Route typology,# Minutes with 1+ VP per Minute,# Minutes with 2+ VP per Minute,Aggregate Actual Service Minutes,Aggregate Scheduled Service Minutes (all trips),# VP,# VP within Scheduled Shape,# Early Arrival Trips,# On-Time Trips,# Late Trips,# Trips with VP,Average VP per Minute,% VP within Scheduled Shape,pct_rt_journey_atleast1_vp,pct_rt_journey_atleast2_vp,% Scheduled Trip w/ 1+ VP/Minute,% Scheduled Trip w/ 2+ VP/Minute,Realtime versus Scheduled Service Ratio,Average Actual Service (Trip Minutes),GTFS Availability,Speed (MPH),route_long_name,route_short_name,Route,Route ID,Base64 Encoded Feed URL,Organization ID,Organization,District,Direction,schedule_source_record_id,Transit Operator,ruler_100_pct,ruler_for_vp_per_min,headway_in_minutes
511,c499f905e33929a641f083dad55c521e,0.00,all_day,31.40,4.43,63,2.62,0.00,0.00,0.00,0.00,1.00,0.00,2025-01-15,downtown_local,3130,3084,5913.36,1978.00,8870,8870,0,1,62,63,1.50,100.00,53.00,52.00,100.00,100.00,2.99,93.86,schedule_and_vp,12.69,E. 14th St. - Mission,10,10 E. 14th St. - Mission,10,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZWVkcz9vcGVyYXRvcl9pZD1BQw==,recOZgevYf7Jimm9L,Alameda-Contra Costa Transit District,04 - Oakland,Eastbound,recJjD8JT53sK302o,Bay Area 511 AC Transit Schedule,100,2,22.90
554,c499f905e33929a641f083dad55c521e,1.00,all_day,35.52,6.64,66,2.75,0.00,0.00,0.00,0.00,1.00,0.00,2025-01-15,downtown_local,3251,3172,4583.45,2344.00,9215,9174,0,5,61,66,2.01,100.00,71.00,69.00,100.00,100.00,1.96,69.45,schedule_and_vp,12.65,E. 14th St. - Mission,10,10 E. 14th St. - Mission,10,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZWVkcz9vcGVyYXRvcl9pZD1BQw==,recOZgevYf7Jimm9L,Alameda-Contra Costa Transit District,04 - Oakland,Westbound,recJjD8JT53sK302o,Bay Area 511 AC Transit Schedule,100,2,21.82


In [47]:
ac_jan_only.Period.value_counts()

all_day    199
Name: Period, dtype: int64

### Timeliness only has `all_day` values.

In [30]:
timeliness_test = _section2_utils.timeliness_trips(sf_only)

In [32]:
timeliness_test.head(2)

,Date,Organization,Route,Period,Direction,dir_0_1,variable,value
0,2023-04-12,City and County of San Francisco,1 CALIFORNIA,offpeak,Westbound,0.00,# Early Arrival Trips,2
1,2023-05-17,City and County of San Francisco,1 CALIFORNIA,offpeak,Westbound,0.00,# Early Arrival Trips,0


In [33]:
timeliness_test.Date.unique()

array(['2023-04-12T00:00:00.000000000', '2023-05-17T00:00:00.000000000',
       '2023-06-14T00:00:00.000000000', '2023-07-12T00:00:00.000000000',
       '2023-08-15T00:00:00.000000000', '2023-10-11T00:00:00.000000000',
       '2023-11-15T00:00:00.000000000', '2023-12-13T00:00:00.000000000',
       '2024-01-17T00:00:00.000000000', '2024-02-14T00:00:00.000000000',
       '2024-03-13T00:00:00.000000000', '2024-04-17T00:00:00.000000000',
       '2024-05-22T00:00:00.000000000', '2024-07-17T00:00:00.000000000',
       '2024-08-14T00:00:00.000000000', '2024-09-18T00:00:00.000000000',
       '2024-10-16T00:00:00.000000000', '2024-11-13T00:00:00.000000000',
       '2024-12-11T00:00:00.000000000', '2023-03-15T00:00:00.000000000'],
      dtype='datetime64[ns]')

In [15]:
timeliness_to_keep = [
    "Date",
    "Organization",
    "Direction",
    "Period",
    "Route",
    "# Early Arrival Trips",
    "# On-Time Trips",
    "# Late Trips",
    "dir_0_1",
]

In [36]:
sf_jan_only[timeliness_to_keep].head(10)

,Date,Organization,Direction,Period,Route,# Early Arrival Trips,# On-Time Trips,# Late Trips,dir_0_1
19,2025-01-15,City and County of San Francisco,Westbound,all_day,1 CALIFORNIA,1,19,169,0.00
77,2025-01-15,City and County of San Francisco,Eastbound,all_day,1 CALIFORNIA,1,18,173,1.00
135,2025-01-15,City and County of San Francisco,Eastbound,all_day,12 FOLSOM-PACIFIC,2,7,75,0.00
193,2025-01-15,City and County of San Francisco,Westbound,all_day,12 FOLSOM-PACIFIC,1,31,53,1.00
251,2025-01-15,City and County of San Francisco,Southbound,all_day,14 MISSION,1,11,128,0.00
309,2025-01-15,City and County of San Francisco,Northbound,all_day,14 MISSION,5,20,122,1.00
348,2025-01-15,City and County of San Francisco,Westbound,all_day,14R MISSION RAPID,4,16,109,0.00
425,2025-01-15,City and County of San Francisco,Northbound,all_day,14R MISSION RAPID,3,8,122,1.00
483,2025-01-15,City and County of San Francisco,Southbound,all_day,18 46TH AVENUE,0,0,48,0.00
541,2025-01-15,City and County of San Francisco,Northbound,all_day,18 46TH AVENUE,0,0,48,1.00


In [21]:
sf_jan_only.Period.value_counts()

all_day    126
Name: Period, dtype: int64

### Timeliness metrics is located in `rt_segment_speeds/segment_speed_utils/metrics.py`

### None of the "route identifiers" are showing up for January 2025

In [ ]:
df.loc[df.service_date == "2025-01-15T00:00:00.000000000"].head()

In [ ]:
df.columns

In [ ]:
df_test.info()

In [ ]:
df_test.route_id = df_test.route_id.fillna("No Route Name")

In [ ]:
df_test.columns

In [ ]:
df_test.service_date.unique()

In [ ]:
df_test.loc[
    (df_test.organization_name == "Monterey-Salinas Transit")
    & (df_test.service_date == "2025-01-15T00:00:00.000000000")
][["route_id"]].nunique()

In [ ]:
# Define a function to highlight the string "No Route Name" in red
def highlight_no_route_name(val):
    color = "red" if val == "No Route Name" else ""
    return f"color: white; background-color: {color}" if color else ""

In [ ]:
january_only = df_test.loc[(df_test.service_date == "2025-01-15T00:00:00.000000000")]

In [ ]:
no_route_names = (
    january_only.groupby(["schedule_gtfs_dataset_key", "organization_name", "route_id"])
    .agg({"direction_id": "count"})
    .reset_index()
)

In [ ]:
# Apply the highlighting function to the 'Route Name' column
no_route_names.style.applymap(highlight_no_route_name, subset=["route_id"])

### Test: removing `name` from the `merge` with `df_crosswalk` in line 259 in `gtfs_digest/merge_data`

### Find out where it's not merging

In [ ]:
df_sched = merge_data.concatenate_schedule_by_route_direction(analysis_date_list)

In [ ]:
df_sched.service_date.unique()

In [ ]:
df_sched.loc[df_sched.service_date == "2025-01-15T00:00:00.000000000"].head(2)

In [ ]:
df_avg_speeds = merge_data.concatenate_speeds_by_route_direction(analysis_date_list)

In [ ]:
df_avg_speeds.loc[df_avg_speeds.service_date == "2025-01-15T00:00:00.000000000"].head(2)

In [ ]:
df_rt_sched = merge_data.concatenate_rt_vs_schedule_by_route_direction(
    analysis_date_list
)

In [ ]:
df_rt_sched.loc[df_rt_sched.service_date == "2025-01-15T00:00:00.000000000"].head(2)

In [ ]:
df_crosswalk = merge_data.concatenate_crosswalk_organization(analysis_date_list)

In [ ]:
df_crosswalk.loc[df_crosswalk.service_date == "2025-01-15T00:00:00.000000000"].head(2)

In [ ]:
primary_typology = merge_data.set_primary_typology(df_sched)

In [ ]:
primary_typology.head(2)

In [ ]:
df_schedule2 = pd.merge(
    df_sched, primary_typology, on=merge_data.route_time_cols, how="left"
)

In [ ]:
df_schedule2.loc[df_schedule2.service_date == "2025-01-15T00:00:00.000000000"].head(2)

In [ ]:
df1 = pd.merge(
    df_schedule2,
    df_rt_sched,
    on=merge_data.route_time_cols + ["service_date"],
    how="outer",
    indicator="sched_rt_category",
).merge(
    df_avg_speeds,
    on=merge_data.route_time_cols + ["service_date"],
    how="outer",
)

In [ ]:
df1.loc[df.service_date == "2025-01-15T00:00:00.000000000"].head(2)

In [ ]:
df1.info()

In [ ]:
df1.sched_rt_category.unique()

In [ ]:
df_wo_standardized_route = df1.assign(
    sched_rt_category=df1.sched_rt_category.map(
        gtfs_schedule_wrangling.sched_rt_category_dict
    )
)

In [ ]:
df_w_standardized_route = df_wo_standardized_route.pipe(
    merge_data.merge_in_standardized_route_names,
)

In [ ]:
df_w_standardized_route.info()

#### `merge_in_standardized_route_names` isn't working
* `route_names_df` doesn't have any values for January 2025.
* Rerun `gtfs_funnel/clean_route_naming.py`

In [ ]:
keep_cols = [
    "schedule_gtfs_dataset_key",
    "name",
    "route_id",
    "service_date",
]

CLEAN_ROUTES = GTFS_DATA_DICT.schedule_tables.route_identification

route_names_df = pd.read_parquet(f"{SCHED_GCS}{CLEAN_ROUTES}.parquet")

In [ ]:
route_names_df.info()

In [ ]:
route_names_df.service_date.unique()

In [ ]:
from segment_speed_utils import gtfs_schedule_wrangling, time_series_utils

In [ ]:
route_names_df = time_series_utils.clean_standardized_route_names(
    route_names_df
).drop_duplicates()

In [ ]:
route_names_df.head(2)

In [ ]:
route_names_df.service_date.unique()

In [ ]:
if "name" in df_wo_standardized_route.columns:
    df_wo_standardized_route = df.drop(columns="name")